In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import psycopg2
from dotenv import load_dotenv
import os
import time

# Web Scrape Stats

In [2]:
def get_player_stats(first,last):
    """
    Returns stats and height of player obtained from sportsreference.com.
    """
    url = f'https://www.sports-reference.com/cbb/players/{first.lower()}-{last.lower()}-1.html'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract basic player info (height, weight, birth date)
    player_info = soup.find_all('p')
    height = None
    for info in player_info:
        if 'cm' in info.text and 'kg' in info.text:  # Height and weight are usually mentioned with these units
            height = info.text.strip()

    # Extracting tables for per game stats and advanced stats
    tables = soup.find_all('table')
    stats = {}
    for table in tables:
        if table.get('id') in ['players_per_game', 'players_advanced']:
            df = pd.read_html(str(table))[0]
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.droplevel(0)  # Dropping multi-level column headers only if they exist
            stats[table.get('id')] = df
            
    if stats != {}:
        try:
            players_per_game = stats.get('players_per_game')
            players_advanced = stats.get('players_advanced')

            pg_cols = ["G", "MP", "FG", "FGA", "FG%", "3P", "3PA", "3P%", "FT", "FTA", "FT%", "TRB", "AST", "STL", "BLK", "TOV", "PF", "PTS", "SOS"]
            adv_cols = ["PER", "TS%", "eFG%", "3PAr", "FTr", "PProd", "ORB%", "DRB%", "TRB%", "AST%", "STL%", "BLK%", "TOV%", "USG%", "OWS", "DWS", "OBPM", "DBPM"]

            players_per_game = players_per_game.loc[:,pg_cols]
            players_advanced = players_advanced.loc[:,adv_cols]
            return height, players_per_game, players_advanced
        except:
            pass
    return height, None, None

def get_player_information(first, last):
    """
    Obtains players date of birth from basketballreference.com.
    """
    url = f'https://www.basketball-reference.com/players/{last[0].lower()}/{last[:5].lower() + first[:2].lower()}01.html'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    element_with_date_birth = soup.find('span', attrs={'data-birth': True})

    # Extract the 'data-birth' attribute
    date_birth = element_with_date_birth['data-birth'] if element_with_date_birth else None

    return date_birth

def get_draft_class(year):
    """
    Returns a list of player names from the given NBA draft year.
    """
    url = f"https://www.basketball-reference.com/draft/NBA_{year}.html"
    response = requests.get(url)
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing the player names
    # Assuming the player names are in 'td' tags with 'data-stat' attribute set to 'player'
    player_name_cells = soup.find_all('td', {'data-stat': 'player'})

    # Extract and print the player names
    player_names = [cell.get_text() for cell in player_name_cells]
    return player_names

def get_player_stats_from_url(url):
    """
    Takes in a url from sportsreference.com and returns per game and advanced stats.
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract basic player info (height, weight, birth date)
    player_info = soup.find_all('p')
    height = None
    for info in player_info:
        if 'cm' in info.text and 'kg' in info.text:  # Height and weight are usually mentioned with these units
            height = info.text.strip()

    # Extracting tables for per game stats and advanced stats
    tables = soup.find_all('table')
    stats = {}
    for table in tables:
        if table.get('id') in ['players_per_game', 'players_advanced']:
            df = pd.read_html(str(table))[0]
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.droplevel(0)  # Dropping multi-level column headers only if they exist
            stats[table.get('id')] = df
            
    if stats != {}:
        try:
            players_per_game = stats.get('players_per_game')
            players_advanced = stats.get('players_advanced')

            pg_cols = ["G", "MP", "FG", "FGA", "FG%", "3P", "3PA", "3P%", "FT", "FTA", "FT%", "TRB", "AST", "STL", "BLK", "TOV", "PF", "PTS", "SOS"]
            adv_cols = ["PER", "TS%", "eFG%", "3PAr", "FTr", "PProd", "ORB%", "DRB%", "TRB%", "AST%", "STL%", "BLK%", "TOV%", "USG%", "OWS", "DWS", "OBPM", "DBPM"]

            players_per_game = players_per_game.loc[:,pg_cols]
            players_advanced = players_advanced.loc[:,adv_cols]
            return height, players_per_game, players_advanced
        except:
            pass
    return height, None, None

def get_birthday_from_url(url):
    """
    Takes in a url from basketballreference.com and returns players birthday
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    element_with_date_birth = soup.find('span', attrs={'data-birth': True})

    # Extract the 'data-birth' attribute
    date_birth = element_with_date_birth['data-birth'] if element_with_date_birth else None

    return date_birth

In [ ]:
get_birthday_from_url("https://www.basketball-reference.com/players/t/tokotjp01.html")

In [ ]:
get_player_stats_from_url("https://www.sports-reference.com/cbb/players/jp-tokoto-1.html")

# SQL functions

In [11]:
def sql_connection():
    """
    Creates a connection to the postgresql server
    """
    
    load_dotenv()

    # Accessing variables from .env file
    db_name = os.getenv('DB_NAME')
    db_user = os.getenv('DB_USER')
    db_pass = os.getenv('DB_PASS')
    db_host = os.getenv('DB_HOST')
    db_port = os.getenv('DB_PORT')
    
    try:
        conn = psycopg2.connect(
            dbname=db_name,
            user=db_user,
            password=db_pass,
            host=db_host,
            port=db_port
        )
        print("Connected to the database.")
        return (conn, conn.cursor())
    except Exception as e:
        print("Unable to connect to the database.")
        print(e)
        return None

def insert_player(conn, cursor, player_tuple):
    """
    Inserts a row into the Player table
    
    cursor : SQL connection cursor
    player_tuple : <tuple> contains player information
    
    Returns the created ID of the player
    """
    insert_statement = """
    INSERT INTO "Players" ("PlayerName", "DOB", "Rating", "DraftYear") VALUES (%s, %s, %s, %s) RETURNING "PlayerID";
    """
    try:
        cursor.execute(insert_statement, player_tuple)
        generated_id = cursor.fetchone()[0]  # Fetch the generated ID
        conn.commit()
        print(player_tuple[0])
        print("Added Player Successfully")
        return generated_id
    except Exception as e:
        print("Unable to insert to the Players Table.")
        print(e)
        return None
    
def insert_stats(conn, cursor, stat_tuple):
    """
    Inserts a row into the PlayerStats table
    
    conn : SQL connection
    cursor : SQL connection cursor
    stat_dict : <tuple> contains stat to be insert into the table
    """
    insert_statement = """
    INSERT INTO "PlayerStats" ("PlayerID", "Games", "MPG", "FG", "FGA", "FGP", "3P", "3PA", "3PP", "FT", "FTA", "FTP", "RPG", "APG", "SPG", "BPG", "TOV", "PF", "PPG", "SOS", "PER", "TS", "eFG", "3PAr", "FTr", "PProd", "ORBP", "DRBP", "TRBP", "ASTP", "STLP", "BLKP", "TOVP", "USG", "OWS", "DWS", "OBPM", "DBPM") VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) RETURNING "StatID";
    """
    
    try:
        cursor.execute(insert_statement, stat_tuple)
        generated_id = cursor.fetchone()[0]  # Fetch the generated ID
        conn.commit()
        print("Added Stat Successfully")
        return generated_id
    except Exception as e:
        print("Unable to insert to the Stats Table.")
        print(e)
        print("Data tuple:", stat_tuple)
        return None

def get_all_players(cursor):
    """
    Returns all players in the database
    """
    query = 'SELECT * FROM "Players"'
    cursor.execute(query)
    records = cursor.fetchall()
    for record in records:
        print(record)

def get_all_stats(cursor):
    """
    Returns all players in the database
    """
    query = 'SELECT * FROM "PlayerStats"'
    cursor.execute(query)
    records = cursor.fetchall()
    for record in records:
        print(record)
        
def update_player_rating(cursor, player_id, rating):
    """
    Updates a players rating in the database
    """
    query = f'UPDATE "Players" SET "Rating" = {rating} WHERE "PlayerID" = {player_id};'
    try:
        cursor.execute(query)
        print("Successfully updated rating")
    except Exception as e:
        print("Failed to update rating")
        print(e)

def update_ratings(conn, cursor, df):
    """
    Updates ratings for players in the database based on those stored in the df
    """
    for index, row in df.iterrows():
        update_player_rating(cursor, row["PlayerID"], row["Rating"])
    conn.commit()
    conn.close()

In [4]:
conn, cursor = sql_connection()

Connected to the database.


In [5]:
def find_names(fullname):
    """
    Takes in a full name and returns a tuple of the first and last name in lower case
    """
    space = fullname.index(" ")
    first = fullname[:space].lower()
    last = fullname[space+1:].lower()
    return (first, last)

def fill_draft_class(conn, cursor, year):
    """
    Fills the database with players from the given draft year
    """
    inserted = []
    player_names = get_draft_class(year)
    for player in player_names:
        print("Attempting to add " + player)
        
        first, last = find_names(player)
        height, players_per_game, players_advanced = get_player_stats(first, last)
        time.sleep(5)
        birth_date = get_player_information(first, last)
        time.sleep(5)
        
        if height is not None and players_per_game is not None and players_advanced is not None and birth_date is not None:
            player_tuple = (player, birth_date, 0, year)
            playerID = insert_player(conn, cursor, player_tuple)
            
            if playerID is not None:
                
                stat_tuple = tuple([playerID] + players_per_game.iloc[-2].tolist() + players_advanced.iloc[-2].tolist())
                insert_stats(conn, cursor, stat_tuple)
                
                inserted.append(player)
                
    return inserted

def fill_drafts(conn, cursor, start, end):
    """
    Fills the database with information on players from draft classes start-end
    """
    players = []
    for draft_year in range(start, end+1):
        print(draft_year)
        draft_players = fill_draft_class(conn, cursor, draft_year)
        players += draft_players
        time.sleep(5)
    
    print("end")
    conn.close()
    return players

def insert_player_stats_url(cursor, player, stat_url, player_url, rating, year):
    """
    Inserts into database from urls
    """
    height, players_per_game, players_advanced = get_player_stats_from_url(stat_url)
    birth_date = get_birthday_from_url(player_url)

    if height is not None and players_per_game is not None and players_advanced is not None and birth_date is not None:
        player_tuple = (player, birth_date, rating, year)
        playerID = insert_player(conn, cursor, player_tuple)
        print("Inserted " + player)
        if playerID is not None:

            stat_tuple = tuple([playerID] + players_per_game.iloc[-2].tolist() + players_advanced.iloc[-2].tolist())
            insert_stats(conn, cursor, stat_tuple)
            print("Inserted " + player + " stats")

def fill_database_from_df(conn, cursor, df):
    """
    Takes in a dataframe and fills from the dataframe
    """
    for index, row in df.iterrows():
        insert_player_stats_url(cursor, row['PlayerName'], row['SportsRef'], row['BballRef'], row['Rating'], row['DraftYear'])
        time.sleep(5)
    conn.commit()
    conn.close()     

In [ ]:
names = fill_drafts(conn, cursor, 2017, 2019)

In [15]:
query = 'SELECT COUNT(*) FROM "Players";'
cursor.execute(query)
records = cursor.fetchall()
print(records)

[(298,)]


In [ ]:
import csv
with open('output.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Write each item as a new row
    for pid, name in records:
        writer.writerow([pid, name])

In [12]:
df = pd.read_csv('output.csv')
update_ratings(conn,cursor,df)

Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated rating
Successfully updated

AttributeError: 'psycopg2.extensions.cursor' object has no attribute 'commit'